## Find active wildfires in UK

With the advent of climate change, and global weather patterns changing the UK is experiencing wildfires increasingly. Some natural, but others due to drier conditions leading to human accididents. 

I wanted to explore using web services to create a visual of fires in UK.

In [1]:
import geopandas as gpd
from geopandas.tools import sjoin

from datetime import datetime
import os
import numpy as np
import pandas as pd
from folium import LayerControl
from IPython.display import Markdown as md
import warnings
import requests
import io
import folium
from folium.plugins import Fullscreen
import os
from shapely.geometry import Point
import zipfile
warnings.filterwarnings("ignore")

In [2]:
### Set folder location for outputs
folder_date = './'+datetime.today().strftime('%Y-%m-%d')
OUTPATH = folder_date
try:
   os.mkdir(OUTPATH)
except Exception as e:
   print('Folder already exists, please check the path')
   raise(e)

In [3]:
def download_zipfile(url, OUTPATH):
    """Function extracts zipfile hosted from defined URL
    returns shape file as geodf"""
    r = requests.get(url)
    z = zipfile.ZipFile(io.BytesIO(r.content))
    filename = url.split('/')[-1].split('.')[0]
    z.extractall(os.path.join(OUTPATH, f'{filename}'))
    shp_file = os.path.join(OUTPATH, f'{filename}', f'{filename}.shp')

    return gpd.read_file(shp_file, crs='EPSG:4326')

def gb_fires(df):
    """Function filters active fires in EU to those in GB"""
    #  filter to bounds of GB grid
    x0, x1 = (-9.0,  2.01) 
    y0, y1 = (49.75, 61.01)
    gb_fires = df[(df.LONGITUDE > x0) & (df.LONGITUDE < x1)]
    gb_fires = gb_fires[(gb_fires.LATITUDE > y0) & (gb_fires.LATITUDE < y1)]
    gb_fires = gb_fires.reset_index(drop=True)
    gb_fires = gb_fires.to_crs('EPSG:27700')
    return gb_fires

def buffer_geometry(df, distance):
    """Function buffers fire points to create an area of fire spread"""
    buffered_df = df.copy()
    buffered_df.geometry = buffered_df.geometry.buffer(distance)
    
    return buffered_df

In [4]:
url = 'https://firms.modaps.eosdis.nasa.gov/data/active_fire/modis-c6.1/shapes/zips/MODIS_C6_1_Europe_48h.zip'
fires = download_zipfile(url, OUTPATH)

In [5]:
fires

,LATITUDE,LONGITUDE,BRIGHTNESS,SCAN,TRACK,ACQ_DATE,ACQ_TIME,SATELLITE,CONFIDENCE,VERSION,BRIGHT_T31,FRP,DAYNIGHT,geometry
0,51.03601,2.28933,301.70,1.49,1.20,2024-04-22,0211,A,41,6.1NRT,276.67,16.30,N,POINT (2.28933 51.03601)
1,48.28131,14.33407,306.65,1.54,1.22,2024-04-22,0211,A,69,6.1NRT,273.95,23.95,N,POINT (14.33407 48.28131)
2,41.15751,0.99878,305.18,1.27,1.12,2024-04-22,0215,A,63,6.1NRT,279.74,14.49,N,POINT (0.99878 41.15751)
3,35.80751,-0.25349,311.00,1.26,1.11,2024-04-22,0217,A,81,6.1NRT,284.36,19.00,N,POINT (-0.25349 35.80751)
4,63.88420,-22.38643,307.34,1.51,1.21,2024-04-22,0525,A,53,6.1NRT,274.85,22.98,N,POINT (-22.38643 63.88420)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,39.01672,-1.88631,311.56,1.00,1.00,2024-04-24,1021,T,39,6.1NRT,299.72,5.81,D,POINT (-1.88631 39.01672)
89,38.61228,-0.58749,310.93,1.02,1.01,2024-04-24,1021,T,44,6.1NRT,299.41,5.58,D,POINT (-0.58749 38.61228)
90,38.95306,-4.75156,305.81,1.18,1.08,2024-04-24,1021,T,38,6.1NRT,294.18,5.21,D,POINT (-4.75156 38.95306)
91,38.66981,-4.05185,308.95,1.10,1.05,2024-04-24,1021,T,39,6.1NRT,298.25,5.74,D,POINT (-4.05185 38.66981)


In [6]:
## Filter the fire events in Europe to boundary of UK (Might capture RI, but drop those)
active_fires = gb_fires(fires)

## Buffer the GB fire points by a defined distance - turn into a polygon to capture some 'fire spread'
#active_fires = buffer_geometry(active_fires, 400.0)

In [7]:
active_fires

,LATITUDE,LONGITUDE,BRIGHTNESS,SCAN,TRACK,ACQ_DATE,ACQ_TIME,SATELLITE,CONFIDENCE,VERSION,BRIGHT_T31,FRP,DAYNIGHT,geometry
0,52.60559,-6.80042,306.45,1.18,1.08,2024-04-23,1117,T,64,6.1NRT,287.47,10.06,D,POINT (75101.350 311695.985)
1,52.61346,-6.79999,310.03,1.13,1.06,2024-04-23,1403,A,69,6.1NRT,292.20,11.20,D,POINT (75188.800 312568.675)


In [8]:
def generate_basemap():
    
    # location point
    centre_of_uk = (53.82, -2.41)
    
    # base map
    m = folium.Map(location=centre_of_uk,
                   tiles='OpenStreetMap',
                   zoom_start=7,
                   control_scale=True,
                   prefer_canvas=True)
    
    # add full screen button
    Fullscreen(title='Expand me',
               title_cancel='Exit fullscreen',
               force_separate_button=True).add_to(m)
    
    return m


def fwas_geojson(fires, m):
    
    #initialise feature group layer
    layer = folium.FeatureGroup(name='VIIRS 48h fire locations')
    style = {'fillColor': '#e6550d', 'color': '#e6550d'}
    gjson = folium.GeoJson(fires,
                           style_function=lambda x: style,
                           zoom_on_click=True)   
    layer.add_child(gjson)
    # add layer to map
    layer.add_to(m)

In [9]:
m = generate_basemap()
fwas_geojson(active_fires, m)
m